# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на том же корпусе но в другую сторону - с русского на английский.
Можно использовать как основу первый или второй способ реализации (с MultiheadAttention или с nn.Transformer). Подберите несколько тестовых примеров для проверки обучения на каждой эпохе.

Параметры ниже точно работают в колабе и модель обучается достаточно быстро. Попробуйте их немного увеличить (batch size возможно придется наоборот уменьшить). Обучайте модель хотя бы 5 эпох, а желательно больше, чтобы тестовые примеры начали переводиться более менее адекватно.

После обучения возьмите хотя бы 100 примером из тестовой части параллельного корпуса и переведите их. Оцените качество переводов с помощью метрики BLEU (пример использования ниже)
Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [EOS] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах! Функция с batch prediction должна работать быстрее, поэтому переведите всю тестовую выборку и оцените качество BLEU на всех данных.

In [ ]:
# В колабе установите torchtune и torchao, чтобы семинарская тетрадка работала
!pip install torchtune torchao

In [ ]:
# пример использования BLEU
# обратите внимание что текты должны быть токенизированы
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room'] # замените на перевод вашей модели
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room'] # замените на эталонный перевод


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

In [ ]:
# параметры которые работают в колабе
embed_dim = 32
num_heads = 4
ff_dim = embed_dim*2
num_layers = 2
batch_size = 400

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers import decoders

import os
import numpy as np
import nltk
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from collections import Counter

from torchtune.modules import RotaryPositionalEmbeddings
from torch.nn import Transformer
import matplotlib.pyplot as plt
%matplotlib inline

Данные:

In [3]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2025-04-01 15:46:39--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  20.8MB/s    in 7.0s    

2025-04-01 15:46:47 (16.6 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2025-04-01 15:46:47--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  17.0MB/s    in 4.5s    

2025-04-01 15:46:52 (14.3 MB/s) - ‘opus.en-ru

In [4]:
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [5]:
ru_sents = open('opus.en-ru-train.ru').read().splitlines()
en_sents = open('opus.en-ru-train.en').read().splitlines()

In [6]:
embed_dim = 32
num_heads = 4
ff_dim = embed_dim*2
num_layers = 2
batch_size = 400
max_len_ru, max_len_en = 47, 48
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[PAD]"], end_of_word_suffix='</w>')
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru)

tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"], end_of_word_suffix='</w>')
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en)

In [8]:
tokenizer_ru.decoder = decoders.BPEDecoder()
tokenizer_en.decoder = decoders.BPEDecoder()

In [9]:
# tokenizer_ru.save('tokenizer_ru')
# tokenizer_en.save('tokenizer_en')

In [10]:
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")
tokenizer_en = Tokenizer.from_file("tokenizer_en")

In [11]:
PAD_IDX = tokenizer_en.token_to_id('[PAD]')
# BOS_IDX = tokenizer_en.token_to_id('[BOS]')
# EOS_IDX = tokenizer_en.token_to_id('[EOS]')
PAD_IDX

0

In [13]:
def encode(text, tokenizer, max_len, encoder=False):
    if encoder:
        return tokenizer.encode(text).ids[:max_len]
    else:
        return [tokenizer.token_to_id('[BOS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[EOS]')]

In [14]:
X_ru = [encode(t, tokenizer_ru, max_len_ru, encoder=True) for t in ru_sents]
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]

In [15]:
X_ru_train, X_ru_valid, X_en_train, X_en_valid = train_test_split(X_ru, X_en, test_size=0.05)

In [16]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, texts_ru, texts_en):
        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, batch_first=True, padding_value=PAD_IDX)

        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, batch_first=True, padding_value=PAD_IDX)

        self.length = len(texts_ru)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        ids_ru = self.texts_ru[index]
        ids_en = self.texts_en[index]
        return ids_ru, ids_en

In [17]:
training_set = Dataset(X_ru_train, X_en_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True)

valid_set = Dataset(X_ru_valid, X_en_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

Трансформер:

In [18]:
class TransformerEncoderDecoder(nn.Module):
    def __init__(self, vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.embedding_enc = nn.Embedding(vocab_size_enc, embed_dim)
        self.embedding_dec = nn.Embedding(vocab_size_dec, embed_dim)
        self.positional_encoding = RotaryPositionalEmbeddings(embed_dim // num_heads, max_seq_len=128)

        self.transformer = Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )

        self.output_layer = nn.Linear(embed_dim, vocab_size_dec)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src_embedded = self.embedding_enc(src)
        B,S,E = src_embedded.shape
        src_embedded = self.positional_encoding(src_embedded.view(B,S,self.num_heads, E//self.num_heads)).view(B,S,E)

        tgt_embedded = self.embedding_dec(tgt)
        B,S,E = tgt_embedded.shape
        tgt_embedded = self.positional_encoding(tgt_embedded.view(B,S,self.num_heads, E//self.num_heads)).view(B,S,E)

        tgt_mask = (~torch.tril(torch.ones((S, S), dtype=torch.bool))).to(DEVICE)

        encoder_output = self.transformer.encoder(
            src_embedded,
            src_key_padding_mask=src_key_padding_mask
        )

        decoder_output = self.transformer.decoder(
            tgt_embedded,
            encoder_output,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask
        )

        output = self.output_layer(decoder_output)
        return output

In [19]:
vocab_size_enc = tokenizer_ru.get_vocab_size()
vocab_size_dec = tokenizer_en.get_vocab_size()

In [20]:
model = TransformerEncoderDecoder(vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers)
model = model.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

In [21]:
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

2.95288 M parameters


In [22]:
def train(model, iterator, optimizer, criterion, scheduler, print_every=100):
    epoch_loss = []
    model.train()

    for i, (texts_ru, texts_en) in enumerate(iterator):
        texts_ru = texts_ru.to(DEVICE)
        texts_en = texts_en.to(DEVICE)
        texts_en_input = texts_en[:,:-1].to(DEVICE)
        texts_en_out = texts_en[:, 1:].to(DEVICE)
        src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
        tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)

        logits = model(texts_ru, texts_en_input, src_padding_mask, tgt_padding_mask)
        optimizer.zero_grad()
        B,S,C = logits.shape
        loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss.append(loss.item())

        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')

    return np.mean(epoch_loss)

def evaluate(model, iterator, criterion):
    epoch_loss = []
    model.eval()
    with torch.no_grad():
        for i, (texts_ru, texts_en) in enumerate(iterator):
            texts_ru = texts_ru.to(DEVICE)
            texts_en = texts_en.to(DEVICE)
            texts_en_input = texts_en[:,:-1].to(DEVICE)
            texts_en_out = texts_en[:, 1:].to(DEVICE)
            src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
            tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)

            logits = model(texts_ru, texts_en_input, src_padding_mask, tgt_padding_mask)

            B,S,C = logits.shape
            loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
            epoch_loss.append(loss.item())

    return np.mean(epoch_loss)

Функция перевода:

In [23]:
@torch.no_grad
def translate(text):
    input_ids = tokenizer_ru.encode(text).ids[:max_len_ru]
    output_ids = [tokenizer_en.token_to_id('[BOS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)], batch_first=True).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], batch_first=True).to(DEVICE)

    src_padding_mask = (input_ids_pad == PAD_IDX).to(DEVICE)
    tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

    logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)

    pred = logits.argmax(2).item()

    while pred not in [tokenizer_en.token_to_id('[EOS]'), tokenizer_en.token_to_id('[PAD]')] and len(output_ids) < 100:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], batch_first=True).to(DEVICE)
        tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

        logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)
        pred = logits.argmax(2).view(-1)[-1].item()

    return tokenizer_en.decoder.decode([tokenizer_en.id_to_token(i) for i in output_ids[1:]])

Обучение:

In [25]:
from timeit import default_timer as timer

NUM_EPOCHS = 10
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, pct_start=0.05,
                                              steps_per_epoch=len(training_generator), epochs=NUM_EPOCHS)

losses = []
test_examples = [
    "Да, но не совсем...",
    "Привет, как дела?",
    "Это пример перевода.",
    "Я изучаю машинное обучение.",
    "Трансформеры - это мощные модели."
]

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(model, training_generator, optimizer, loss_fn, scheduler)
    end_time = timer()
    val_loss = evaluate(model, valid_generator, loss_fn)

    if not losses or val_loss < min(losses):
        print(f'Improved to {val_loss}, saving model..')
        torch.save(model, 'model')

    losses.append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
          f"Epoch time={(end_time-start_time):.3f}s"))

    # Print test examples
    print("\nTest translations:")
    for example in test_examples:
        print(f"RU: {example}")
        print(f"EN: {translate(example)}\n")

Loss: 10.355045833587646;
Loss: 10.167172012329102;
Loss: 9.935965979894002;
Loss: 9.628733744621277;
Loss: 9.24804971408844;
Loss: 8.881004736423492;
Loss: 8.586942828042167;
Loss: 8.33937436044216;
Loss: 8.124135661125184;
Loss: 7.935873209953308;
Loss: 7.76853129863739;
Loss: 7.620374661684036;
Loss: 7.488636261866643;
Loss: 7.3691024879046845;
Loss: 7.260065105438232;
Loss: 7.16180088698864;
Loss: 7.071753857276019;
Loss: 6.988029187255435;
Loss: 6.911399175242374;
Loss: 6.840401908874512;
Loss: 6.7749978633154;
Loss: 6.713493946032091;
Loss: 6.656047990425773;


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


Improved to 5.283758491516113, saving model..
Epoch: 1, Train loss: 6.615, Val loss: 5.284, Epoch time=418.501s

Test translations:
RU: Да, но не совсем...
EN: No , no , but I ' m not not not not not not not not not .

RU: Привет, как дела?
EN: You ' re going ?

RU: Это пример перевода.
EN: It ' s a good .

RU: Я изучаю машинное обучение.
EN: I ' m a little .

RU: Трансформеры - это мощные модели.
EN: We ' re gonna be a little .

Loss: 5.3318988227844235;
Loss: 5.314874973297119;
Loss: 5.297300187746684;
Loss: 5.284473857879639;
Loss: 5.272793460845947;
Loss: 5.264507575829824;
Loss: 5.2547317280088155;
Loss: 5.2438782924413685;
Loss: 5.232617939313252;
Loss: 5.221666922092438;
Loss: 5.211262934424661;
Loss: 5.2012636586030325;
Loss: 5.191612660334661;
Loss: 5.181989494051252;
Loss: 5.17279104423523;
Loss: 5.164083296954632;
Loss: 5.155127008662505;
Loss: 5.146764573785994;
Loss: 5.139076070534555;
Loss: 5.131712389230728;
Loss: 5.123841428756714;
Loss: 5.11631867538799;
Loss: 5.109080

Метрика BLEU:

In [26]:
ru_test = open('opus.en-ru-test.ru').read().replace('\xa0', ' ').splitlines()[:100]
en_test = open('opus.en-ru-test.en').read().splitlines()[:100]

In [28]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [29]:
bleu_scores = []
translations = []

for ru, en_ref in zip(ru_test, en_test):
    translation = translate(ru)
    translations.append((ru, en_ref, translation))

    hyp_tokens = nltk.word_tokenize(translation.lower())
    ref_tokens = nltk.word_tokenize(en_ref.lower())

    bleu = nltk.translate.bleu_score.sentence_bleu([ref_tokens], hyp_tokens, auto_reweigh=True)
    bleu_scores.append(bleu)

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [30]:
top_indices = np.argsort(bleu_scores)[-5:][::-1]
print("\nTop 5 translations by BLEU score:")
for idx in top_indices:
    ru, en_ref, en_hyp = translations[idx]
    print(f"\nRussian: {ru}")
    print(f"Reference EN: {en_ref}")
    print(f"Translated EN: {en_hyp}")
    print(f"BLEU: {bleu_scores[idx]:.4f}")

print(f"\nAverage BLEU score: {np.mean(bleu_scores):.4f}")


Top 5 translations by BLEU score:

Russian: Да.
Reference EN: - Yes.
Translated EN: Yes .
BLEU: 0.6065

Russian: Я тебя люблю, папа.
Reference EN: I love you, Dad. How great!
Translated EN: I love you , Dad .
BLEU: 0.6065

Russian: Now, if you have kids yourself, you know what--
Reference EN: Now, if you have kids yourself, you know what--
Translated EN: You know , you know what you ' re gonna be a little bit of the world .
BLEU: 0.1340

Russian: Если вы хотите завершить этот процесс, вам понадобится минимум 8-10 лет, если не хотите рисковать тем, чтобы Пауни стал...
Reference EN: If you want to see this through, you need eight to ten years minimum, unless you want to run the risk of Pawnee becoming...
Translated EN: If you want to be a few years , I want to be a few years , I don ' t think that it ' t to be a lot of the time .
BLEU: 0.0964

Russian: b) что касается основного имущества, утраченного или поврежденного в результате единичного враждебного действия или вынужденного оставле

Переводы не очень, а один из примеров, который должен быть на русском, почему-то на английском...

Но для очень простых предложений перевод довольно хороший (как для "да" и "я тебя люблю, папа"). Во втором примере странно, что в английском есть еще добавление "How great!", а в русском ничего подобного нет. Возможно в корпусе больше таких случаев, что в теории может влиять на качество модели.

С увеличением количества эпох модель выдавала лучшие переводы для контрольных примеров. Если модель еще пообучать, качество тоже улучшилось бы


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

Back translation - это техника, которая используется в машинном переводе для аугментации данных. Она применяется, когда данных в параллельном корпусе мало, например, когда мы имеем дело с малоресурсными языками, или когда какой-то определенный жанр плохо представлен в датасете, но при этом у нас есть монолингвальные данные.

Сначала мы используем параллельный корпус для обучения модели машинного перевода в обратном направлении (изначально мы хотим переводить из языка-источника в язык-таргет) - мы переводим из языка-таргета в язык-источник (a target-to-source MT system). Затем мы используем эту модель для перевода монолингвального корпуса (языка-таргета) в язык-источник.

После этого мы объединяем естественно полученные данные с искусственными и обучаем модель, которая будет нам переводить язык-источник в язык-таргет.



---



Теперь разберем, как применить эту технику к данным из семинара.

Языком-источником является английский, языком-таргетом - русский.

У нас есть маленький пареллельный англо-русский корпус и много монолингвальных данных на русском.

Нам понадобится 2 модели (одна переводит с английского на русский, вторая - с русского на английский).

И нам надо проделать следующие шаги:


1.   Обучаем модель, которая переводит в обратном направлении (с русского на английский) на параллельном корпусе.
2.   При помощи этой же модели получаем больше параллельных данных, используя ее при переводе монолингвального корпуса (он на русском языке).
1.   Объединяем естественно полученные данные и синтетические данные в один большой датасет.
2.   Обучаем новую модель, которая переводит с языка-источника на язык-таргет, то есть с английского на русский. Мы улучшаем качество модели путем увеличения данных (мы обучаем модель на большом параллельном корпусе).

Таким образом, необходимо будет сделать 3 запуска модели.